#  Using TorchServe on SageMaker Inf2.24xlarge with LLAMAv2-13B


## Step 1: Let's bump up SageMaker and import stuff

The wheel installed here is a private preview wheel, you need to add into allowlist to run this function

In [ ]:
!python --version

In [ ]:
# Install the latest aws cli v2 if it is not installed
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip awscliv2.zip
#Follow the instruction to install aws v2 on the terminal
!cat aws/README.md

In [ ]:
# Conform it is aws-cli/2.xx.xx
!aws --version

In [ ]:
#%pip install sagemaker pip --upgrade  --quiet
!pip install numpy
!pip install pillow
!pip install -U sagemaker
!pip install -U boto 
!pip install -U botocore
!pip install -U boto3

In [1]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

boto3_session=boto3.session.Session(region_name="us-west-2")
smr = boto3.client('sagemaker-runtime-demo')
sm = boto3.client('sagemaker')
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess= sagemaker.session.Session(boto3_session, sagemaker_client=sm, sagemaker_runtime_client=smr)  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

# Configuration:
bucket_name = sess.default_bucket()
prefix = "torchserve"
output_path = f"s3://{bucket_name}/{prefix}"
print(f'account={account_id}, region={region}, role={role}, output_path={output_path}')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
account=084495728311, region=us-west-2, role=arn:aws:iam::084495728311:role/service-role/AmazonSageMaker-ExecutionRole-20230505T104760, output_path=s3://sagemaker-us-west-2-084495728311/torchserve


## Step 2: Build a BYOD TorchServe Docker container and push it to Amazon ECR

In [ ]:
# Install our own dependencies
!cat workspace/docker/Dockerfile

In [ ]:
%%capture build_output

baseimage = "763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference-neuronx:1.13.1-neuronx-py310-sdk2.12.0-ubuntu20.04"
reponame = "neuronx212"
versiontag = "latest"

# Build our own docker image
!cd workspace/docker && ./build_and_push.sh {reponame} {versiontag} {baseimage} {region} {account}

In [ ]:
if 'Error response from daemon' in str(build_output):
    print(build_output)
    raise SystemExit('\n\n!!There was an error with the container build!!')
else:
    container = str(build_output).strip().split('\n')[-1]

print(container)

## Step 3: AOT Pre-Compile Model on EC2 (Optional in this example)

This step is to precompile model with batchSize and save it in cache dir to use it in later model loading. It can significantly reduce model loading latency to a few seconds.

#### Precompile the model at local EC2

Follow [Instruction](https://github.com/pytorch/serve/pull/2458/files#diff-bc416c811f749ead11ab8f100d5a4198fa453adc995b0760272563971638307d) at local EC2 to precompile the model. The precompiled model will be stored in `model_store/llama-2-13b/neuron_cache`

#### Main steps on EC2
                     
* download llama-2-13b from HF 
* Save the model split checkpoints compatible with `transformers-neuronx`
* Create EC2 model artifacts at local

#### Keynotes

1. **Turn on neuron_cache** in function initialize of inf2_handler.py
```
os.environ["NEURONX_CACHE"] = "on"
os.environ["NEURONX_DUMP_TO"] = f"{model_dir}/neuron_cache"
```

2. Set **micro_batch_size: 4** in model-config.yaml to precompile the model with batch size 4 in this example
```
minWorkers: 1
maxWorkers: 1
maxBatchDelay: 100
responseTimeout: 10800
batchSize: 16

handler:
    amp: "bf16"
    tp_degree: 6
    max_length: 100

micro_batching:
    micro_batch_size: 4
    parallelism:
        preprocess: 2
        inference: 1
        postprocess: 2
```

3. The generated cache files are **tightly coupled with neuronx SDK version**
```
This example is based on torch-neuronx==1.13.1.1.9.0
```

4. Create **SageMaker model artifacts format** once the model artifacts is generated on EC2
```
cd model_store/llama-2-13b/
mkdir code
mv requirements.txt code/
```

## Step 4: Upload model artifacts to S3

You can choose either option 1 or option 2.

* option 1: Sample command to copy llama-2-13b from EC2 to S3
```
print(output_path)
!aws s3 cp llama-2-13b {output_path}/llama-2-13b --recursive
s3_uri = f"{output_path}/llama-2-13b-neuronx-b4/"
```

* option 2: The model artifacts of this example is available in s3://torchserve/mar_files/sm-neuronx/llama-2-13b-neuronx-b4/ which supports llama2-13b on neuronx batchSize = 4 based on torch-neuronx==1.13.1.1.9.0. You can copy it to your SM S3 model repo. For example

In [ ]:
!aws s3 cp s3://torchserve/mar_files/sm-neuronx/llama-2-13b-neuronx-b4/ llama-2-13b-neuronx-b4 --recursive

In [ ]:
!aws s3 cp llama-2-13b-neuronx-b4 {output_path}/llama-2-13b-neuronx-b4 --recursive

In [2]:
s3_uri = f"{output_path}/llama-2-13b-neuronx-b4/"
print(s3_uri)

s3://sagemaker-us-west-2-084495728311/torchserve/llama-2-13b-neuronx-b4/


## Step 5: Start building SageMaker endpoint
In this step, we will build SageMaker endpoint from scratch

### Create SageMaker endpoint

You need to specify the instance to use and endpoint names

In [5]:
from datetime import datetime

instance_type = "ml.inf2.24xlarge"
endpoint_name = sagemaker.utils.name_from_base("ts-inf2-llama2-13b-b1")

model = Model(
    name="torchserve-inf2-llama2-13b" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S"),
    model_data={
        "S3DataSource": {
                "S3Uri": s3_uri,
                "S3DataType": "S3Prefix",
                "CompressionType": "None",
        }
    },
    image_uri=f"084495728311.dkr.ecr.us-west-2.amazonaws.com/neuronx212:latest",
    #image_uri=container,
    role=role,
    sagemaker_session=sess,
)
print(model)

In [6]:
model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    volume_size=512, # increase the size to store large model
    model_data_download_timeout=3600, # increase the timeout to download large model
    container_startup_health_check_timeout=3600, # ncrease the timeout to load large model
)

Your model is not compiled. Please compile your model before using Inferentia.


-------------!

## Run the inference

In [7]:
import io


class Parser:
    """
    A helper class for parsing the byte stream input. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'scan_lines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        data = self.buff.getvalue()
        
    def scan_lines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

In [8]:
import json

body = "Today the weather is really nice and I am planning on".encode('utf-8')
resp = smr.invoke_endpoint_with_response_stream(EndpointName=endpoint_name, Body=body, ContentType="application/json")
event_stream = resp['Body']
parser = Parser()
for event in event_stream:
    parser.write(event['PayloadPart']['Bytes'])
    for line in parser.scan_lines():
        print(line.decode("utf-8"), end=' ')

Today the weather is really nice and I am planning on going to the beach. I am going to take my camera and take some pictures. I am going to take pictures of the beach and the ocean. I am going to take pictures of the people and the animals. I am going to take pictures of the sun and the sky. I am going to take pictures of the sand and the water. I am going to take pictures of the trees and the flowers. I am going to take pictures of the birds and the butterflies. I a 

## Clean up the environment

In [ ]:
sess.delete_endpoint(endpoint_name)
sess.delete_endpoint_config(endpoint_name)
model.delete_model()